In [2]:
import pandas as pd
import numpy as np
from sql_engine import connect

In [3]:
conn_scerp = connect("new_ERP")
conn_scdrp = connect("drp")
conn_bi = connect("bidata")

In [4]:
from pyecharts import options as opts
from pyecharts.charts import Bar,Line,Grid,Pie,Funnel
from pyecharts.faker import Faker
from pyecharts.globals import ThemeType
from pyecharts.commons.utils import JsCode

In [5]:
sql_customer = """
SELECT
	a.UnionNumber,
	a.RealName,
	a.AgentLevelName,
	CONVERT ( VARCHAR ( 10 ),a.UpTime, 23 ) AS UpTime,
CASE
		a.LowerStatus 
		WHEN 1 THEN
		'正常' 
		WHEN 2 THEN
		'降级' 
		WHEN 3 THEN
		'清退' 
		WHEN 4 THEN
		'冻结' ELSE '待补充' 
	END AS LowerStatus,
	d.official_name as TopUserName,
	c.AllianceName,
	b.Level,
	a.RegionalDirectorName,
	a.RegionManagerName,
	a.CustomerServiceName,
	a.CustomerServiceAreaName,
	a.CustomerServiceManagerName 
FROM
	b_UnionUserInfo_FromDRP a
	LEFT JOIN bidata.bidata.dbo.Active_Official_Relation d ON a.TopUserID = d.official_UN
LEFT JOIN ( SELECT
	a.unionno,
	a.level 
FROM
	bidata.bidata.dbo.WS_Customer_Level a,
	( SELECT unionno, MAX ( Version ) as version FROM bidata.bidata.dbo.WS_Customer_Level GROUP BY unionno ) b	
WHERE
	a.unionno = b.unionno 
	AND a.Version = b.version ) b on a.UnionNumber = b.unionno
LEFT JOIN
	bidata.bidata.dbo.IMP_Alter_Alliancename c ON a.AllianceName = c.AllianceName_full
WHERE
	( LowerStatus < 5 OR LowerStatus IS NULL ) 
	
"""

In [6]:
sql_customer_balance = """SELECT
	c.unionnumber,
	SUM ( CAST ( amt_total AS DECIMAL ( 18, 2 ) ) ) / 1000000 as balance 
FROM
	SCLM_DRPDB.dbo.[A_SybBalanceStock] a
	LEFT JOIN bidata.bidata.dbo.ZCLX b ON a.activityid = b.zc_id
	LEFT JOIN SCLM_DRPDB.[dbo].b_UserInfo c ON a.userid = c.f_id 
WHERE
	CreateTime = ( SELECT MAX ( CreateTime ) FROM SCLM_DRPDB.[dbo].[A_SybBalanceStock] ) 
	AND b.zctype = '混拿回款' 
	AND amt_total != 0 
GROUP BY
	c.unionnumber"""

In [7]:
year1="2019"
year2="2021"
month1,month2 = "03","03"
var = "7"
sql_returned_ship_year = f"""SELECT 
	UnionNumber,
	sellername,
	month,
	sum(returned_money) as returned_money,
	sum(ship_money) as ship_money
FROM
(
SELECT
	UnionNumber,
	sellername,
	CONVERT ( VARCHAR ( {var} ), DateNum, 23 ) AS MONTH,
	SUM ( returned_money + JK_returned_money ) AS returned_money,
	SUM ( money + JK_money ) AS ship_money 
FROM
	bidata.bidata.dbo.InterMediate_WS_agent_zc_daily a
	LEFT JOIN bidata.bidata.dbo.ZCLX b ON a.zc_id = b.zc_id 
WHERE
	DateNum BETWEEN '{year1}-{month1}-01' AND '{year2}-{month2}-31' 

	AND b.zctype = '混拿回款' 
GROUP BY
	UnionNumber,sellername,
	CONVERT ( VARCHAR ( {var} ), DateNum, 23 )
	-- bi每天备份的回款数据，备份的是前一天
UNION ALL

SELECT
	b.UnionNumber,
	a.sellername,
	CONVERT ( VARCHAR ( {var} ), GETDATE(), 23 ) AS MONTH,
	SUM ( flowprice ) AS money,
	0 AS ship_money
FROM
	(
	SELECT
		agentID,
		sellername,
		SUM ( ( InValue - OutValue ) / 10000 ) AS flowprice 
	FROM
		dbo.DH_BalanceApply a
		-- 三草ERP回款
		LEFT JOIN [bidata].[bidata].dbo.ZCLX b 	ON a.ActivityID = b.zc_id
	WHERE
		zctype = '混拿回款'
		AND status = 1 
		AND Deleted = 0 
		AND (
			Item IN ( '货物预存款', '客户余额互转', '余额清理', '政策余额充值', '政策余额转款', '三合一转款', '全品牌转出','全品牌A转B', '代理清算', '卡位收支', '全品牌转入', '全品牌转款' ) 
			OR ( Item IN ( '政策赠送', '退赠送金额' ) AND ActivityName LIKE '%意向金%' ) 
		) 
		AND CONVERT(varchar(10),auditime, 120 ) between '{year1}-{month1}-01' AND '{year2}-{month2}-31'
		AND DATEDIFF(dd, auditime, getdate()) = 0

	GROUP BY
		agentID,sellername
	UNION ALL
	SELECT
		agentID,
		sellername,
		SUM ( ( InValue - OutValue ) / 10000 ) AS flowprice 
	FROM
		JKERPDB_UE.dbo.DH_BalanceApply a 
		-- 健康ERP回款
		LEFT JOIN [bidata].[bidata].dbo.ZCLX b ON a.ActivityID = b.zc_id 
	WHERE
		zctype = '混拿回款'
		AND status = 1 
		AND Deleted = 0 
		AND (
			Item IN ( '货物预存款', '客户余额互转', '余额清理', '政策余额充值', '政策余额转款', '三合一转款', '全品牌转出','全品牌A转B', '代理清算', '卡位收支', '全品牌转入', '全品牌转款' ) 
			OR ( Item IN ( '政策赠送', '退赠送金额' ) AND ActivityName LIKE '%意向金%' ) 
		) 
		AND CONVERT(varchar(10),auditime, 120 ) between '{year1}-{month1}-01' AND '{year2}-{month2}-31'
		AND DATEDIFF(dd, auditime, getdate()) = 0

	GROUP BY
		agentID,sellername
	) a
	LEFT JOIN dbo.DH_UserInfo b ON a.agentid = b.id
GROUP BY
	b.UnionNumber,a.sellername
UNION ALL

-- =====================发货款
SELECT
	b.UnionNumber,
	a.sellername,
	CONVERT ( VARCHAR ( {var} ), GETDATE(), 23 ) AS MONTH,
	0 as money,
	SUM ( money ) AS ship_money 
FROM
	(
	SELECT
		a.agentid,
		e.sellername,
		SUM ( ( d.totalAmount- isnull( d.apportionAmount, 0 ) ) / totalCount * b.count ) / 10000 AS money 
	FROM
		dbo.DH_Order_Shipment a
		LEFT JOIN dbo.DH_Order_Shipments_Relation b ON a.shipNo= b.shipNo
		LEFT JOIN dbo.DH_Order c ON a.orderNo= c.orderNo AND b.orderNo= c.orderNo
		LEFT JOIN dbo.DH_Order_Detail d ON c.orderNo= d.orderNo AND b.orderDetailNo= d.orderDetailNo
		LEFT JOIN bidata.bidata.dbo.ZCLX e ON c.activityid = e.zc_id 
	WHERE
		a.status= 80 
		AND len( a.agentid ) > 0 
		AND a.isDeleted= 0 
		AND b.isDeleted= 0 
		AND c.orderstatus > 200 
		AND len( c.orderNo ) > 0 
		AND e.zctype = '混拿回款' 
		AND CONVERT ( VARCHAR ( 10 ), a.sendtime, 23 ) BETWEEN '{year1}-{month1}-01' AND '{year2}-{month2}-31'
		AND DATEDIFF(dd, a.sendtime, getdate()) = 0
	GROUP BY
		a.agentid,e.sellername
		
-- ==========================ERP发货款
	UNION ALL
	
	--==================减去退货款
	SELECT
		b.AgentId,
		c.sellername,
		SUM ( drawbackvalue ) /- 10000 AS money 
	FROM
		(
		SELECT
			AfterNo,
			drawbackvalue 
		FROM
			dbo.DH_AfterSale_Details 
		WHERE
			refuntype = '退货' 
			AND CONVERT ( VARCHAR ( 10 ), auditeTimeStorage, 120 ) BETWEEN '{year1}-{month1}-01' AND '{year2}-{month2}-31'  
			AND DATEDIFF(dd, auditeTimeStorage, getdate()) = 0
		) a
		LEFT JOIN ( SELECT agentId, AfterNo, activityid FROM dbo.DH_AfterSale WHERE refundFlag = 88 ) b ON a.AfterNo = b.AfterNo
		LEFT JOIN bidata.bidata.dbo.ZCLX c ON b.activityid = c.zc_id 
	WHERE
		zctype = '混拿回款' 

	GROUP BY
		b.AgentId,c.sellername
	) a
	LEFT JOIN dbo.DH_UserInfo b ON a.agentid = b.id 
GROUP BY
	b.UnionNumber,a.sellername
	) a
	GROUP BY
		UnionNumber,sellername,
		month
ORDER BY
	month,
	UnionNumber"""

# 1. 回款分析


## 1.1 年周期回款分析

#### 1.1.1 数据准备

**1) 基础表提取**
- 提取代理信息数据
- 提取余额数据
- 提取年周期回款发货金额数据


In [8]:
df_customer_info = pd.read_sql(sql_customer,conn_scerp)
df_customer_balance = pd.read_sql(sql_customer_balance,conn_scdrp)
df_returned_ship_year = pd.read_sql(sql_returned_ship_year,conn_scerp)

In [9]:
df_customer_info.head()

,UnionNumber,RealName,AgentLevelName,UpTime,LowerStatus,TopUserName,AllianceName,Level,RegionalDirectorName,RegionManagerName,CustomerServiceName,CustomerServiceAreaName,CustomerServiceManagerName
0,CM00001068,冯桥娥,全品牌总代,2018-02-25,正常,郑会连,黑马,B类,李海艳,官露,郭思梦,崔颖,张妍
1,CM00001252,胡红梅,全品牌总代,2018-02-26,正常,孙兴宏,女王,C类,官瑞,徐元江,陈琼,郑雄文,余芬芬
2,CM00003028,蒋娜,全品牌总代,2019-02-14,正常,禹明鑫,明星,C类,官瑞,徐元江,胡冰,鲜于开迪,余芬芬
3,CM00002402,但佳,护肤总代,2016-05-03,清退,袁芬,卓彩,None,李海艳,陈思,王弯,肖建,徐莉
4,CM00005035,王聪颖,养面膜总代,2019-06-29,正常,王超,卓彩,None,李海艳,陈思,胡慧敏,焦慧,徐莉


In [10]:
df_customer_balance.head()

,unionnumber,balance
0,CM00006198,2.285200
1,CM00003105,6.076860
2,CM00001235,12.906200
3,CM00004969,0.049400
4,CM00002122,0.464022


In [11]:
df_returned_ship_year['month'] = pd.to_datetime(df_returned_ship_year.month,format='%Y-%m')
df_returned_ship_year.dtypes

UnionNumber               object
sellername                object
month             datetime64[ns]
returned_money           float64
ship_money               float64
dtype: object

- 调整回款数据时间序列

In [12]:
df_returned_ship_year.describe()

,returned_money,ship_money
count,81029.000000,81029.000000
mean,1.873276,1.872689
std,5.004243,3.338462
min,-58.243600,-11.704400
25%,0.000000,0.237100
50%,0.000000,0.872300
75%,2.000000,2.250000
max,180.000000,180.360000


**2) 合并代理信息表和年周期余额发货数据表**

In [13]:
df_customer_returned_ship_year = pd.merge(df_customer_info,df_returned_ship_year,'left',on = 'UnionNumber')
df_customer_returned_ship_year.head()

,UnionNumber,RealName,AgentLevelName,UpTime,LowerStatus,TopUserName,AllianceName,Level,RegionalDirectorName,RegionManagerName,CustomerServiceName,CustomerServiceAreaName,CustomerServiceManagerName,sellername,month,returned_money,ship_money
0,CM00001068,冯桥娥,全品牌总代,2018-02-25,正常,郑会连,黑马,B类,李海艳,官露,郭思梦,崔颖,张妍,三草两木,2019-03-01,2.0000,1.6216
1,CM00001068,冯桥娥,全品牌总代,2018-02-25,正常,郑会连,黑马,B类,李海艳,官露,郭思梦,崔颖,张妍,三草两木,2019-04-01,-0.9388,3.9472
2,CM00001068,冯桥娥,全品牌总代,2018-02-25,正常,郑会连,黑马,B类,李海艳,官露,郭思梦,崔颖,张妍,三草两木全品牌,2019-04-01,9.7867,3.7888
3,CM00001068,冯桥娥,全品牌总代,2018-02-25,正常,郑会连,黑马,B类,李海艳,官露,郭思梦,崔颖,张妍,三草两木,2019-05-01,0.0000,0.1800
4,CM00001068,冯桥娥,全品牌总代,2018-02-25,正常,郑会连,黑马,B类,李海艳,官露,郭思梦,崔颖,张妍,三草两木全品牌,2019-05-01,0.0753,0.9948


In [14]:
df_customer_returned_ship_year.describe()

,returned_money,ship_money
count,80670.000000,80670.000000
mean,1.880729,1.877698
std,5.009138,3.309794
min,-58.243600,-11.704400
25%,0.000000,0.240050
50%,0.000000,0.878400
75%,2.000000,2.250000
max,180.000000,180.360000


In [15]:
df_customer_returned_ship_year.dtypes

UnionNumber                           object
RealName                              object
AgentLevelName                        object
UpTime                                object
LowerStatus                           object
TopUserName                           object
AllianceName                          object
Level                                 object
RegionalDirectorName                  object
RegionManagerName                     object
CustomerServiceName                   object
CustomerServiceAreaName               object
CustomerServiceManagerName            object
sellername                            object
month                         datetime64[ns]
returned_money                       float64
ship_money                           float64
dtype: object

In [16]:
df_customer_returned_ship_year.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 81543 entries, 0 to 81542
Data columns (total 17 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   UnionNumber                 81543 non-null  object        
 1   RealName                    81543 non-null  object        
 2   AgentLevelName              81279 non-null  object        
 3   UpTime                      80502 non-null  object        
 4   LowerStatus                 81543 non-null  object        
 5   TopUserName                 80941 non-null  object        
 6   AllianceName                81084 non-null  object        
 7   Level                       67089 non-null  object        
 8   RegionalDirectorName        80989 non-null  object        
 9   RegionManagerName           80989 non-null  object        
 10  CustomerServiceName         81541 non-null  object        
 11  CustomerServiceAreaName     81542 non-null  object    

In [17]:
df_customer_returned_ship_year.isnull().sum()

UnionNumber                       0
RealName                          0
AgentLevelName                  264
UpTime                         1041
LowerStatus                       0
TopUserName                     602
AllianceName                    459
Level                         14454
RegionalDirectorName            554
RegionManagerName               554
CustomerServiceName               2
CustomerServiceAreaName           1
CustomerServiceManagerName        1
sellername                      873
month                           873
returned_money                  873
ship_money                      873
dtype: int64

In [18]:
df_customer_returned_ship_year.isnull().any()

UnionNumber                   False
RealName                      False
AgentLevelName                 True
UpTime                         True
LowerStatus                   False
TopUserName                    True
AllianceName                   True
Level                          True
RegionalDirectorName           True
RegionManagerName              True
CustomerServiceName            True
CustomerServiceAreaName        True
CustomerServiceManagerName     True
sellername                     True
month                          True
returned_money                 True
ship_money                     True
dtype: bool

#### 1.1.2 年周期回款发货数据

1) 按月聚合

In [19]:
df_returned_ship_year_gy = df_customer_returned_ship_year.groupby('month')[['returned_money','ship_money']].sum()
df_returned_ship_year_gy

,returned_money,ship_money
month,,
2019-03-01,5998.9628,7466.1105
2019-04-01,14544.0383,10395.5248
2019-05-01,1611.6110,5976.9112
2019-06-01,9871.4899,10651.6393
2019-07-01,476.7838,4324.1858
2019-08-01,156.6492,2771.0794
2019-09-01,23150.3702,13327.2100
2019-10-01,8886.6700,9210.5764
2019-11-01,6424.6829,5833.2421


In [20]:
s1 = df_returned_ship_year_gy.copy().reset_index()
s2 = df_returned_ship_year_gy.diff(12).reset_index()
s = s1.merge(s2, on='month', how='inner')
s['yoy_return'] = s['returned_money_y'] /(s['returned_money_x'] -s['returned_money_y'])
s['yoy_ship'] = s['ship_money_y'] /(s['ship_money_x'] -s['ship_money_y'])
s.yoy_return = s.yoy_return.apply(lambda x: format(x, '.2%'))
s.yoy_ship = s.yoy_ship.apply(lambda x: format(x, '.2%'))
s = s[s.returned_money_y.notnull()][['month','returned_money_x','yoy_return','ship_money_x','yoy_ship']]
s.reset_index(drop = True)

,month,returned_money_x,yoy_return,ship_money_x,yoy_ship
0,2020-03-01,17877.8875,198.02%,4118.6279,-44.84%
1,2020-04-01,10992.0695,-24.42%,23654.6372,127.55%
2,2020-05-01,7125.1951,342.12%,4585.9066,-23.27%
3,2020-06-01,21.9286,-99.78%,3340.0414,-68.64%
4,2020-07-01,1245.3771,161.20%,2747.2665,-36.47%
5,2020-08-01,1405.1547,797.01%,2835.9645,2.34%
6,2020-09-01,1130.4859,-95.12%,1980.8331,-85.14%
7,2020-10-01,11942.8450,34.39%,8744.1362,-5.06%
8,2020-11-01,8377.8352,30.40%,3808.5087,-34.71%
9,2020-12-01,113.1413,-53.90%,3555.0215,-9.09%


In [21]:
s['yoy_ship'] =  s['yoy_ship'].apply(lambda x : '100%' if x =='inf%' else x )
s

,month,returned_money_x,yoy_return,ship_money_x,yoy_ship
12,2020-03-01,17877.8875,198.02%,4118.6279,-44.84%
13,2020-04-01,10992.0695,-24.42%,23654.6372,127.55%
14,2020-05-01,7125.1951,342.12%,4585.9066,-23.27%
15,2020-06-01,21.9286,-99.78%,3340.0414,-68.64%
16,2020-07-01,1245.3771,161.20%,2747.2665,-36.47%
17,2020-08-01,1405.1547,797.01%,2835.9645,2.34%
18,2020-09-01,1130.4859,-95.12%,1980.8331,-85.14%
19,2020-10-01,11942.8450,34.39%,8744.1362,-5.06%
20,2020-11-01,8377.8352,30.40%,3808.5087,-34.71%
21,2020-12-01,113.1413,-53.90%,3555.0215,-9.09%


- 月度维度下总回款和回款同比

In [22]:
# df_returned_ship_year_gy.resample('Q').mean()

### 1.1.3 联盟维度年周期回款发货数据

In [23]:
df_AllianceName_returned_ship_year_gy = df_customer_returned_ship_year.groupby(['month','AllianceName'])[['returned_money','ship_money']].sum()
df_AllianceName_returned_ship_year_gy

returned_money  ship_money
month      AllianceName                            
2019-03-01 BOSS                523.2735    721.0313
           其他                    0.0000      1.0078
           北极星                 103.7992    121.1095
           卓彩                  387.6876    601.7040
           太阳                  184.8731    320.9992
...                                 ...         ...
2021-03-01 满天星                 260.5293    281.2099
           王者                  327.1722    328.0480
           聚微                 1376.5605   1014.5611
           风之彩                 710.6245    550.2470
           黑马                 1621.7766   1326.3719

[316 rows x 2 columns]

In [24]:
def func_(df):
    df = df.groupby('month').sum()
    s1 = df.copy().reset_index()
    s2 = df.diff(12).reset_index()
    s = s1.merge(s2, on='month' , how='inner')
    s['yoy_return'] = s['returned_money_y'] /(s['returned_money_x'] -s['returned_money_y'])
    s['yoy_ship'] = s['ship_money_y'] /(s['ship_money_x'] -s['ship_money_y'])
#     s.yoy_return = s.yoy_return.apply(lambda x: format(x, '.2%'))
#     s.yoy_ship = s.yoy_ship.apply(lambda x: format(x, '.2%'))
    s = s[s.returned_money_y.notnull()][['month','returned_money_x','yoy_return','ship_money_x','yoy_ship']]
    s.reset_index(drop = True)
    return s

In [25]:
a = df_customer_returned_ship_year.groupby('AllianceName')[['month', 'returned_money','ship_money']].apply(func_)
a.reset_index(inplace = True)
a.drop(columns = 'level_1',axis = 1,inplace = True)
a

,AllianceName,month,returned_money_x,yoy_return,ship_money_x,yoy_ship
0,BOSS,2020-03-01,1630.4152,2.115799,343.0139,-0.524273
1,BOSS,2020-04-01,1003.8390,-0.305661,2157.6967,0.932024
2,BOSS,2020-05-01,719.3285,3.840850,534.6602,-0.104618
3,BOSS,2020-06-01,5.1035,-0.994898,385.2714,-0.662092
4,BOSS,2020-07-01,362.7708,12.367632,500.4855,0.448552
...,...,...,...,...,...,...
155,黑马,2020-11-01,952.6528,0.414295,484.0310,-0.211160
156,黑马,2020-12-01,1.1277,-0.972826,402.3098,-0.086515
157,黑马,2021-01-01,434.1391,0.109645,464.6047,0.362619
158,黑马,2021-02-01,17.9678,0.478734,167.7704,inf


#### 一年周期联盟回款排行

In [26]:
a.groupby('AllianceName').returned_money_x.sum().sort_values(ascending = False)

AllianceName
明星      17354.2467
挑战者      9920.8257
黑马       9776.1256
聚微       8408.5169
BOSS     8065.7475
卓彩       6546.3990
女王       4510.4841
风之彩      2994.8020
满天星      2940.4219
王者       2907.3592
太阳       1624.3586
北极星      1150.2446
其他        370.0000
Name: returned_money_x, dtype: float64

#### 前五回款联盟年周期数据

In [27]:
returned_money_x =  a.pivot_table(values=['returned_money_x','yoy_return'],index= 'month',columns='AllianceName')['returned_money_x'].loc[:,['明星','挑战者','黑马','聚微','BOSS']]
returned_money_x['明星']

month
2020-03-01    4128.7339
2020-04-01    2899.7435
2020-05-01    1623.5019
2020-06-01      18.5944
2020-07-01     210.8415
2020-08-01     262.2691
2020-09-01     316.6567
2020-10-01    2740.9321
2020-11-01    1701.4910
2020-12-01      -1.0278
2021-01-01     902.8719
2021-02-01       8.0481
2021-03-01    2541.5904
Name: 明星, dtype: float64

In [28]:
yoy_return = a.pivot_table(values=['returned_money_x','yoy_return'],index= 'month',columns='AllianceName')['yoy_return'].loc[:,['明星','挑战者','黑马','聚微','BOSS']]
yoy_return

AllianceName,明星,挑战者,黑马,聚微,BOSS
month,,,,,
2020-03-01,1.498270,2.511557,3.909741,1.905703,2.115799
2020-04-01,-0.110443,-0.133304,-0.214033,-0.312028,-0.305661
2020-05-01,4.150119,6.669680,2.930588,2.837758,3.840850
2020-06-01,-0.991008,-1.010758,-0.996203,-0.998496,-0.994898
2020-07-01,0.676749,2.316123,7.347688,-0.251743,12.367632
2020-08-01,3.887846,21.588906,3.817970,4.888771,115.219501
2020-09-01,-0.950039,-0.933073,-0.909181,-0.959879,-0.968578
2020-10-01,0.204370,0.390093,0.361833,-0.049499,0.382868
2020-11-01,-0.001475,0.762739,0.414295,-0.146555,0.889168


In [29]:
c = (
    Line(init_opts = opts.InitOpts(width= '1200px',height='600px',theme=ThemeType.LIGHT))
    .add_xaxis(returned_money_x.index.strftime('%Y-%m').tolist())
    .add_yaxis("明星", round(returned_money_x.iloc[:,0]).tolist())
    .add_yaxis("挑战者", round(returned_money_x.iloc[:,1]).tolist())
    .add_yaxis("黑马", round(returned_money_x.iloc[:,2]).tolist())
    .add_yaxis("聚微", round(returned_money_x.iloc[:,3]).tolist())
    .add_yaxis("BOSS", round(returned_money_x.iloc[:,4]).tolist())
    .set_global_opts(title_opts=opts.TitleOpts(title= '联盟环比'),yaxis_opts = opts.AxisOpts(min_= 0,is_show = True),)
    .set_series_opts(
            label_opts=opts.LabelOpts(is_show = False)
    )
)
c.render_notebook()

In [30]:
c = (
    Line(init_opts = opts.InitOpts(width= '1000px',height='400px',theme=ThemeType.WESTEROS))
    .add_xaxis(yoy_return.index.strftime('%Y-%m').tolist())
    .add_yaxis("明星", [round(n*100,2) for n in yoy_return.iloc[:,0].tolist()])
    .add_yaxis("挑战者", [round(n*100,2) for n in yoy_return.iloc[:,1].tolist()])
    .add_yaxis("黑马", [round(n*100,2) for n in yoy_return.iloc[:,2].tolist()])
    .set_global_opts(
            title_opts =opts.TitleOpts(title = '联盟同比'),
            yaxis_opts=opts.AxisOpts(axislabel_opts=opts.LabelOpts(font_size = 12,formatter="{value} %")),
            xaxis_opts=opts.AxisOpts(axislabel_opts=opts.LabelOpts(font_size = 14,formatter="{value} %"),is_show = False)
    )
    .set_series_opts(
            label_opts=opts.LabelOpts(font_size = 14,formatter= JsCode("function (params) {return params.value[1]+'%'}"))
    )
)
c.render_notebook()

In [31]:
bar = (
    Bar()
    .add_xaxis(returned_money_x.index.strftime('%Y-%m').tolist())
    .extend_axis(yaxis=opts.AxisOpts(
        type_="value", position="right",axislabel_opts=opts.LabelOpts(font_size = 14,formatter="{value} %"),
        axisline_opts = opts.AxisLineOpts(is_show = False)))
    .add_yaxis("明星-回款金额", round(returned_money_x.iloc[:,0]).tolist(),yaxis_index=0)
    .add_yaxis("挑战者-回款金额", round(returned_money_x.iloc[:,1]).tolist(),yaxis_index=0)
    .add_yaxis("黑马-回款金额", round(returned_money_x.iloc[:,2]).tolist(),yaxis_index=0)
    .set_global_opts(
        title_opts=opts.TitleOpts(title='明星-回款金额和同比'),
        yaxis_opts = opts.AxisOpts(min_= 0,is_show = True,axislabel_opts=opts.LabelOpts(font_size = 14),axisline_opts = opts.AxisLineOpts(is_show = False)),
        xaxis_opts = opts.AxisOpts(axislabel_opts=opts.LabelOpts(font_size = 12,rotate = -40))
    )
    .set_series_opts(label_opts=opts.LabelOpts(font_size = 14))
                    )
line = (
    Line()
    .add_xaxis(yoy_return.index.strftime('%Y-%m').tolist())
    .add_yaxis("明星-回款同比", [round(n*100,2) for n in yoy_return.iloc[:,0].tolist()],yaxis_index=1,z=10)
    .set_global_opts(
            title_opts =opts.TitleOpts(title = '联盟同比'),
    )
    .set_series_opts(
            label_opts=opts.LabelOpts(font_size = 14,formatter= JsCode("function (params) {return params.value[1]+'%'}"))
    )        
)
bar.overlap(line)
grid = Grid(init_opts = opts.InitOpts(theme=ThemeType.WONDERLAND,width='1200px',height='600px'))
grid.add(bar, opts.GridOpts(pos_left="5%", pos_right="20%"), is_control_axis_index=True)
grid.render_notebook()

#### 前五回款联盟年周期同比数据

In [32]:
a.loc[a['month'] == '2021-03-01'].sort_values('returned_money_x',ascending = False).reset_index(drop = True)

,AllianceName,month,returned_money_x,yoy_return,ship_money_x,yoy_ship
0,明星,2021-03-01,2541.5904,-0.384414,2098.2538,0.681586
1,挑战者,2021-03-01,1935.8947,0.046443,1747.6187,3.418207
2,黑马,2021-03-01,1621.7766,-0.413233,1326.3719,2.190290
3,聚微,2021-03-01,1376.5605,-0.401748,1014.5611,0.992397
4,BOSS,2021-03-01,1219.8000,-0.251847,979.5398,1.855685
5,卓彩,2021-03-01,868.7122,-0.453732,749.2656,0.835080
6,女王,2021-03-01,844.9088,-0.222789,665.8519,1.031238
7,风之彩,2021-03-01,710.6245,0.296995,550.2470,4.169959
8,王者,2021-03-01,327.1722,-0.600710,328.0480,1.254323
9,太阳,2021-03-01,301.8940,0.000464,258.4067,2.834946


- 增加联盟维度的同比

In [33]:
a.groupby('AllianceName')['returned_money_x'].sum().sort_values(ascending = False)[:5]
# 过去一年回款前五的联盟

AllianceName
明星      17354.2467
挑战者      9920.8257
黑马       9776.1256
聚微       8408.5169
BOSS     8065.7475
Name: returned_money_x, dtype: float64

- 过去一年回款前五的联盟：明星、挑战者、黑马、聚微、BOSS

## 1.2 月周期回款分析

### 1.2.1 数据准备

1) 读取今年月回款数据

In [34]:
year1="2021"
year2="2021"
month1,month2 = "01","03"
var = "10"
sql_returned_ship_month = f"""SELECT 
	UnionNumber,
	sellername,
	month,
	sum(returned_money) as returned_money,
	sum(ship_money) as ship_money
FROM
(
SELECT
	UnionNumber,
	sellername,
	CONVERT ( VARCHAR ( {var} ), DateNum, 23 ) AS MONTH,
	SUM ( returned_money + JK_returned_money ) AS returned_money,
	SUM ( money + JK_money ) AS ship_money 
FROM
	bidata.bidata.dbo.InterMediate_WS_agent_zc_daily a
	LEFT JOIN bidata.bidata.dbo.ZCLX b ON a.zc_id = b.zc_id 
WHERE
	DateNum BETWEEN '{year1}-{month1}-01' AND '{year2}-{month2}-31' 

	AND b.zctype = '混拿回款' 
GROUP BY
	UnionNumber,sellername,
	CONVERT ( VARCHAR ( {var} ), DateNum, 23 )
	-- bi每天备份的回款数据，备份的是前一天
UNION ALL

SELECT
	b.UnionNumber,
	a.sellername,
	CONVERT ( VARCHAR ( {var} ), GETDATE(), 23 ) AS MONTH,
	SUM ( flowprice ) AS money,
	0 AS ship_money
FROM
	(
	SELECT
		agentID,
		sellername,
		SUM ( ( InValue - OutValue ) / 10000 ) AS flowprice 
	FROM
		dbo.DH_BalanceApply a
		-- 三草ERP回款
		LEFT JOIN [bidata].[bidata].dbo.ZCLX b 	ON a.ActivityID = b.zc_id
	WHERE
		zctype = '混拿回款'
		AND status = 1 
		AND Deleted = 0 
		AND (
			Item IN ( '货物预存款', '客户余额互转', '余额清理', '政策余额充值', '政策余额转款', '三合一转款', '全品牌转出','全品牌A转B', '代理清算', '卡位收支', '全品牌转入', '全品牌转款' ) 
			OR ( Item IN ( '政策赠送', '退赠送金额' ) AND ActivityName LIKE '%意向金%' ) 
		) 
		AND CONVERT(varchar(10),auditime, 120 ) between '{year1}-{month1}-01' AND '{year2}-{month2}-31'
		AND DATEDIFF(dd, auditime, getdate()) = 0

	GROUP BY
		agentID,sellername
	UNION ALL
	SELECT
		agentID,
		sellername,
		SUM ( ( InValue - OutValue ) / 10000 ) AS flowprice 
	FROM
		JKERPDB_UE.dbo.DH_BalanceApply a 
		-- 健康ERP回款
		LEFT JOIN [bidata].[bidata].dbo.ZCLX b ON a.ActivityID = b.zc_id 
	WHERE
		zctype = '混拿回款'
		AND status = 1 
		AND Deleted = 0 
		AND (
			Item IN ( '货物预存款', '客户余额互转', '余额清理', '政策余额充值', '政策余额转款', '三合一转款', '全品牌转出','全品牌A转B', '代理清算', '卡位收支', '全品牌转入', '全品牌转款' ) 
			OR ( Item IN ( '政策赠送', '退赠送金额' ) AND ActivityName LIKE '%意向金%' ) 
		) 
		AND CONVERT(varchar(10),auditime, 120 ) between '{year1}-{month1}-01' AND '{year2}-{month2}-31'
		AND DATEDIFF(dd, auditime, getdate()) = 0

	GROUP BY
		agentID,sellername
	) a
	LEFT JOIN dbo.DH_UserInfo b ON a.agentid = b.id
GROUP BY
	b.UnionNumber,a.sellername
UNION ALL

-- =====================发货款
SELECT
	b.UnionNumber,
	a.sellername,
	CONVERT ( VARCHAR ( {var} ), GETDATE(), 23 ) AS MONTH,
	0 as money,
	SUM ( money ) AS ship_money 
FROM
	(
	SELECT
		a.agentid,
		e.sellername,
		SUM ( ( d.totalAmount- isnull( d.apportionAmount, 0 ) ) / totalCount * b.count ) / 10000 AS money 
	FROM
		dbo.DH_Order_Shipment a
		LEFT JOIN dbo.DH_Order_Shipments_Relation b ON a.shipNo= b.shipNo
		LEFT JOIN dbo.DH_Order c ON a.orderNo= c.orderNo AND b.orderNo= c.orderNo
		LEFT JOIN dbo.DH_Order_Detail d ON c.orderNo= d.orderNo AND b.orderDetailNo= d.orderDetailNo
		LEFT JOIN bidata.bidata.dbo.ZCLX e ON c.activityid = e.zc_id 
	WHERE
		a.status= 80 
		AND len( a.agentid ) > 0 
		AND a.isDeleted= 0 
		AND b.isDeleted= 0 
		AND c.orderstatus > 200 
		AND len( c.orderNo ) > 0 
		AND e.zctype = '混拿回款' 
		AND CONVERT ( VARCHAR ( 10 ), a.sendtime, 23 ) BETWEEN '{year1}-{month1}-01' AND '{year2}-{month2}-31'
		AND DATEDIFF(dd, a.sendtime, getdate()) = 0
	GROUP BY
		a.agentid,e.sellername
		
-- ==========================ERP发货款
	UNION ALL
	
	--==================减去退货款
	SELECT
		b.AgentId,
		c.sellername,
		SUM ( drawbackvalue ) /- 10000 AS money 
	FROM
		(
		SELECT
			AfterNo,
			drawbackvalue 
		FROM
			dbo.DH_AfterSale_Details 
		WHERE
			refuntype = '退货' 
			AND CONVERT ( VARCHAR ( 10 ), auditeTimeStorage, 120 ) BETWEEN '{year1}-{month1}-01' AND '{year2}-{month2}-31'  
			AND DATEDIFF(dd, auditeTimeStorage, getdate()) = 0
		) a
		LEFT JOIN ( SELECT agentId, AfterNo, activityid FROM dbo.DH_AfterSale WHERE refundFlag = 88 ) b ON a.AfterNo = b.AfterNo
		LEFT JOIN bidata.bidata.dbo.ZCLX c ON b.activityid = c.zc_id 
	WHERE
		zctype = '混拿回款' 

	GROUP BY
		b.AgentId,c.sellername
	) a
	LEFT JOIN dbo.DH_UserInfo b ON a.agentid = b.id 
GROUP BY
	b.UnionNumber,a.sellername
	) a
	GROUP BY
		UnionNumber,sellername,
		month
ORDER BY
	month,
	UnionNumber"""

In [35]:
df_returned_ship_month = pd.read_sql(sql_returned_ship_month,conn_scerp)

2) 关联客户信息表并增加姓名-官方字段

In [36]:
df_customer_returned_ship_month = pd.merge(df_customer_info,df_returned_ship_month,'left',on = 'UnionNumber')
df_customer_returned_ship_month['name_Alliance'] = df_customer_returned_ship_month['RealName'] +'-'+df_customer_returned_ship_month['AllianceName']

In [37]:
df_customer_returned_ship_month.head()

,UnionNumber,RealName,AgentLevelName,UpTime,LowerStatus,TopUserName,AllianceName,Level,RegionalDirectorName,RegionManagerName,CustomerServiceName,CustomerServiceAreaName,CustomerServiceManagerName,sellername,month,returned_money,ship_money,name_Alliance
0,CM00001068,冯桥娥,全品牌总代,2018-02-25,正常,郑会连,黑马,B类,李海艳,官露,郭思梦,崔颖,张妍,三草两木全品牌,2021-01-08,0.0,0.2200,冯桥娥-黑马
1,CM00001068,冯桥娥,全品牌总代,2018-02-25,正常,郑会连,黑马,B类,李海艳,官露,郭思梦,崔颖,张妍,三草两木全品牌,2021-01-09,0.0,0.0840,冯桥娥-黑马
2,CM00001068,冯桥娥,全品牌总代,2018-02-25,正常,郑会连,黑马,B类,李海艳,官露,郭思梦,崔颖,张妍,三草两木全品牌,2021-01-26,0.0,0.1816,冯桥娥-黑马
3,CM00001068,冯桥娥,全品牌总代,2018-02-25,正常,郑会连,黑马,B类,李海艳,官露,郭思梦,崔颖,张妍,三草两木全品牌,2021-01-27,1.0,0.0000,冯桥娥-黑马
4,CM00001068,冯桥娥,全品牌总代,2018-02-25,正常,郑会连,黑马,B类,李海艳,官露,郭思梦,崔颖,张妍,三草两木全品牌,2021-01-28,0.0,0.1680,冯桥娥-黑马


#### 第一季度回款发货数据

In [38]:
df_returned_ship_quarter = df_customer_returned_ship_month.loc[(df_customer_returned_ship_month['month'] >='2021-01-01') & (df_customer_returned_ship_month['month'] <='2021-03-31'),['month','UnionNumber','name_Alliance','AllianceName','returned_money','ship_money']]
df_returned_ship_quarter.head()

,month,UnionNumber,name_Alliance,AllianceName,returned_money,ship_money
0,2021-01-08,CM00001068,冯桥娥-黑马,黑马,0.0,0.2200
1,2021-01-09,CM00001068,冯桥娥-黑马,黑马,0.0,0.0840
2,2021-01-26,CM00001068,冯桥娥-黑马,黑马,0.0,0.1816
3,2021-01-27,CM00001068,冯桥娥-黑马,黑马,1.0,0.0000
4,2021-01-28,CM00001068,冯桥娥-黑马,黑马,0.0,0.1680


In [39]:
df_returned_ship_quarter_gy = df_customer_returned_ship_month.groupby(['UnionNumber','name_Alliance'])[['returned_money','ship_money']].sum().sort_values('returned_money',ascending= False).reset_index()

#### 读取第一季度发货和出货目标

In [40]:
sql_target_return = """SELECT UnionNumber,Target target_return FROM [dbo].[Target_2021_WS_agent_CAL]
where Quarter =1 and type = '回款'"""
sql_target_ship = """SELECT UnionNumber,Target target_ship FROM [dbo].[Target_2021_WS_agent_CAL]
where Quarter =1 and type = '出货'"""

In [41]:
df_target_return = pd.read_sql(sql_target_return,conn_bi)
df_sql_target_ship = pd.read_sql(sql_target_ship,conn_bi)

In [42]:
df_returned_ship_quarter_gy = df_returned_ship_quarter_gy.merge(df_target_return,on='UnionNumber',how='left').merge(df_sql_target_ship,on='UnionNumber',how='left')

In [43]:
df_returned_ship_quarter_gy['returned_rat'] = df_returned_ship_quarter_gy['returned_money'] / df_returned_ship_quarter_gy['target_return']
df_returned_ship_quarter_gy['ship_rat'] = df_returned_ship_quarter_gy['ship_money'] / df_returned_ship_quarter_gy['target_ship']

df_returned_ship_quarter_gy.loc[(df_returned_ship_quarter_gy['returned_money']>0)&(df_returned_ship_quarter_gy['target_return'].isnull()),'returned_rat'] =1
df_returned_ship_quarter_gy['returned_rat'].fillna(-1, inplace=True)
# 筛选回款金额大于0，且回款目标为空的列，为回款率赋值为1。剩下的为空的回款率就是回款金额小于0且回款目标的数据。
df_returned_ship_quarter_gy.loc[(df_returned_ship_quarter_gy['ship_money']>0)&(df_returned_ship_quarter_gy['target_ship'].isnull()),'ship_rat'] =1
df_returned_ship_quarter_gy['ship_rat'].fillna(-1, inplace=True)
# 同回款数据
df_returned_ship_quarter_gy

,UnionNumber,name_Alliance,returned_money,ship_money,target_return,target_ship,returned_rat,ship_rat
0,CM00007651,郑国贤-其他,190.0000,160.0000,NaN,NaN,1.000000,1.000000
1,CM00007650,谢喜艳-其他,180.0000,180.3600,NaN,NaN,1.000000,1.000000
2,CM00003664,毛文-黑马,172.0000,158.2766,17.89,22.23,9.614310,7.119955
3,CM00000095,张雪萍-明星,100.6276,23.8729,28.42,24.57,3.540732,0.971628
4,CM00000067,董全红-明星,100.0110,27.2848,29.40,25.42,3.401735,1.073360
...,...,...,...,...,...,...,...,...
6658,CM00001144,谢纯珍-明星,-6.3004,0.0000,NaN,NaN,-1.000000,-1.000000
6659,CM00006712,赵丹-黑马,-6.7095,0.0000,5.88,5.44,-1.141071,0.000000
6660,CM00004619,龙雪-明星,-6.7981,0.0000,NaN,NaN,-1.000000,-1.000000
6661,CM00004278,寇方玉-女王,-7.3748,0.0000,NaN,NaN,-1.000000,-1.000000


In [44]:
# df_returned_ship_quarter_gy.apply(lambda x: x['returned_money']/x['target_return'] if x['target_return']>0 else np.sign(x['returned_money']), axis=1)
# 用函数方法来计算完成率

In [45]:
df_returned_ship_quarter_gy = df_returned_ship_quarter_gy[df_returned_ship_quarter_gy['target_return'].notnull()].reset_index(drop=True)

In [46]:
df_returned_ship_quarter_gy_top15 =  df_returned_ship_quarter_gy[0:15]
df_returned_ship_quarter_gy_top15

,UnionNumber,name_Alliance,returned_money,ship_money,target_return,target_ship,returned_rat,ship_rat
0,CM00003664,毛文-黑马,172.0000,158.2766,17.89,22.23,9.614310,7.119955
1,CM00000095,张雪萍-明星,100.6276,23.8729,28.42,24.57,3.540732,0.971628
2,CM00000067,董全红-明星,100.0110,27.2848,29.40,25.42,3.401735,1.073360
3,CM00000086,刘亚歌-明星,100.0000,26.5930,28.91,24.99,3.459011,1.064146
4,CM00000105,高朵朵-聚微,100.0000,16.6150,24.50,24.73,4.081633,0.671856
5,CM00000287,郭孟洁-明星,100.0000,14.3762,29.40,25.42,3.401361,0.565547
6,CM00000244,王婷婷-风之彩,100.0000,33.6197,26.46,22.87,3.779289,1.470035
7,CM00000221,刘双华-黑马,100.0000,12.0968,12.74,14.43,7.849294,0.838309
8,CM00000258,余宏敏-BOSS,100.0000,33.8491,34.30,29.65,2.915452,1.141622
9,CM00000569,李燕-挑战者,66.0000,61.2378,17.64,16.61,3.741497,3.686803


In [47]:
df_returned_ship_quarter_gy_top15['name_Alliance'].tolist()
df_returned_ship_quarter_gy_top15['returned_money'].tolist()
df_returned_ship_quarter_gy_top15['ship_money'].tolist()
df_returned_ship_quarter_gy_top15['returned_rat'].tolist()

[9.614309670206818,
 3.54073187895848,
 3.4017346938775512,
 3.459010722933241,
 4.081632653061225,
 3.4013605442176873,
 3.7792894935752077,
 7.849293563579278,
 2.915451895043732,
 3.7414965986394555,
 2.4068641699273337,
 1.4577259475218658,
 2.3211607142857145,
 2.190635451505017,
 2.7251700680272113]

In [48]:
bar = (
    Bar()
    .add_xaxis(df_returned_ship_quarter_gy_top15['name_Alliance'].tolist())
    .extend_axis(yaxis=opts.AxisOpts(
        type_="value", position="right",axislabel_opts=opts.LabelOpts(font_size = 14,formatter="{value} %"),
        axisline_opts = opts.AxisLineOpts(is_show = False),axistick_opts =  opts.AxisTickOpts(is_show = False)))
    .add_yaxis("回款金额", round(df_returned_ship_quarter_gy_top15['returned_money'],2).tolist(),yaxis_index=0,stack="stack1")
    .add_yaxis("发货金额", round(df_returned_ship_quarter_gy_top15['ship_money'],2).tolist(),yaxis_index=0,stack="stack1")
    
    .set_global_opts(
        title_opts=opts.TitleOpts(title="第一季度-Top15-回款代理"),
        yaxis_opts = opts.AxisOpts(min_= 0,is_show = True,axislabel_opts=opts.LabelOpts(font_size = 14),axisline_opts = opts.AxisLineOpts(is_show = False)),
        xaxis_opts = opts.AxisOpts(axislabel_opts=opts.LabelOpts(font_size = 12,rotate = -40))
    )
    .set_series_opts(label_opts=opts.LabelOpts(font_size = 14,is_show = False))
                    )
line = (
    Line()
    .add_xaxis(df_returned_ship_quarter_gy_top15['name_Alliance'].tolist())
    .add_yaxis("回款完成率",[round(n*100,2) for n in df_returned_ship_quarter_gy_top15['returned_rat'].tolist()] ,linestyle_opts= opts.LineStyleOpts(width =1),yaxis_index=1,z=10,label_opts=opts.LabelOpts(font_size = 12,formatter="{c} %"))
    .add_yaxis("发货完成率",[round(n*100,2) for n in df_returned_ship_quarter_gy_top15['ship_rat'].tolist()] ,yaxis_index=1,z=10,label_opts=opts.LabelOpts(font_size = 12,formatter="{c} %"))
    .set_global_opts(
            title_opts =opts.TitleOpts(title = ''),
    )
    .set_series_opts(
            label_opts=opts.LabelOpts(font_size = 14,formatter= JsCode("function (params) {return params.value[1]+'%'}"),is_show= False)
           
    )   )     
bar.overlap(line)
grid = Grid(init_opts = opts.InitOpts(theme=ThemeType.WESTEROS,width='850px',height='600px'))
grid.add(bar, opts.GridOpts(pos_left="5%", pos_right="20%"), is_control_axis_index=True)
grid.render_notebook()

In [49]:
line = (
    Line()
    .add_xaxis(yoy_return.index.strftime('%Y-%m').tolist())
    .add_yaxis("回款完成率",[round(n*100,2) for n in df_returned_ship_quarter_gy_top15['returned_rat'].tolist()] ,yaxis_index=1,z=10)
    .set_global_opts(
            title_opts =opts.TitleOpts(title = '联盟同比'),
    )
    .set_series_opts(
            label_opts=opts.LabelOpts(font_size = 14,formatter= JsCode("function (params) {return params.value[1]+'%'}"))
    )        
)

- 第一季的回款和发货完成率

In [50]:
df =  df_returned_ship_quarter_gy[:5].copy()
df

,UnionNumber,name_Alliance,returned_money,ship_money,target_return,target_ship,returned_rat,ship_rat
0,CM00003664,毛文-黑马,172.0000,158.2766,17.89,22.23,9.614310,7.119955
1,CM00000095,张雪萍-明星,100.6276,23.8729,28.42,24.57,3.540732,0.971628
2,CM00000067,董全红-明星,100.0110,27.2848,29.40,25.42,3.401735,1.073360
3,CM00000086,刘亚歌-明星,100.0000,26.5930,28.91,24.99,3.459011,1.064146
4,CM00000105,高朵朵-聚微,100.0000,16.6150,24.50,24.73,4.081633,0.671856


In [51]:
df['22'] = df['returned_money'] + df['ship_money']
df['33'] = df['returned_money'].sum()
df['44'] = df['returned_money'].cumsum()
df

,UnionNumber,name_Alliance,returned_money,ship_money,target_return,target_ship,returned_rat,ship_rat,22,33,44
0,CM00003664,毛文-黑马,172.0000,158.2766,17.89,22.23,9.614310,7.119955,330.2766,572.6386,172.0000
1,CM00000095,张雪萍-明星,100.6276,23.8729,28.42,24.57,3.540732,0.971628,124.5005,572.6386,272.6276
2,CM00000067,董全红-明星,100.0110,27.2848,29.40,25.42,3.401735,1.073360,127.2958,572.6386,372.6386
3,CM00000086,刘亚歌-明星,100.0000,26.5930,28.91,24.99,3.459011,1.064146,126.5930,572.6386,472.6386
4,CM00000105,高朵朵-聚微,100.0000,16.6150,24.50,24.73,4.081633,0.671856,116.6150,572.6386,572.6386


## 当月的回款发货数据

In [52]:
df_returned_ship_03 = df_customer_returned_ship_month.loc[(df_customer_returned_ship_month['month'] >='2021-03-01') & (df_customer_returned_ship_month['month'] <='2021-03-31'),['month','UnionNumber','name_Alliance','TopUserName','AllianceName','returned_money','ship_money']]
df_returned_ship_03.ship_money.sum()

10251.139000000001

### 每天的回款情况和平均回款差异

In [53]:
def diff(df): 
    df['avg_returned'] = df['returned_money'].mean()
    df['avg_ship'] = df['ship_money'].mean()
    df['diff_returned'] = df['returned_money'] - df['avg_returned']
    df['diff_ship'] = df['ship_money'] - df['avg_ship']
    df.drop(['avg_returned','avg_ship'],axis = 1,inplace = True)
    return df

In [54]:
df_returned_ship_day = df_returned_ship_03.groupby('month')[['returned_money','ship_money']].sum()
df_returned_ship_day.reset_index(inplace = True)
df = diff(df_returned_ship_day)
df

,month,returned_money,ship_money,diff_returned,diff_ship
0,2021-03-01,-2.6134,118.7418,-397.149148,-211.940103
1,2021-03-02,6906.6384,974.7769,6512.102652,644.094997
2,2021-03-03,761.9765,1776.9546,367.440752,1446.272697
3,2021-03-04,639.8398,473.6330,245.304052,142.951097
4,2021-03-05,372.4646,456.7543,-22.071148,126.072397
5,2021-03-06,375.6583,464.1915,-18.877448,133.509597
6,2021-03-07,189.7925,455.7500,-204.743248,125.068097
7,2021-03-08,227.4376,423.2385,-167.098148,92.556597
8,2021-03-09,248.8905,337.7930,-145.645248,7.111097
9,2021-03-10,275.9735,198.8648,-118.562248,-131.817103


In [55]:
bar = (
    Bar()
    .add_xaxis(df['month'].tolist())
    .extend_axis(yaxis=opts.AxisOpts(
        type_="value", position="right",axislabel_opts=opts.LabelOpts(font_size = 14,formatter="{value}"),
        axisline_opts = opts.AxisLineOpts(is_show = False)))
    .add_yaxis("回款较均值差额", round(df['diff_returned'],0).tolist(),yaxis_index=0)
    .add_yaxis("发货较均值差额", round(df['diff_ship'],0).tolist(),yaxis_index=0)
    .set_global_opts(
        title_opts=opts.TitleOpts(title="3月回款发货"),
        yaxis_opts = opts.AxisOpts(is_show = True,axislabel_opts=opts.LabelOpts(font_size = 14),axisline_opts = opts.AxisLineOpts(is_show = False)),
        xaxis_opts = opts.AxisOpts(axislabel_opts=opts.LabelOpts(font_size = 12,rotate = -40))
    )
    .set_series_opts(label_opts=opts.LabelOpts(font_size = 14,is_show = False))
                    )
line = (
    Line()
    .add_xaxis(df['month'].tolist())
    .add_yaxis("回款金额",round(df['returned_money'],0).tolist() ,yaxis_index=1,z=10, markpoint_opts=opts.MarkPointOpts(data=[opts.MarkPointItem(type_="max")]),markline_opts=opts.MarkLineOpts(data=[opts.MarkLineItem(type_="average")]))
    .add_yaxis("发货金额",round(df['ship_money'],0).tolist() ,yaxis_index=1,z=10, markpoint_opts=opts.MarkPointOpts(data=[opts.MarkPointItem(type_="max")]))
    .set_global_opts(
            title_opts =opts.TitleOpts(title = ''),
    )
    .set_series_opts(
            label_opts=opts.LabelOpts(font_size = 14,formatter= JsCode("function (params) {return params.value[1]+'%'}"),is_show= False)
    )        
)
bar.overlap(line)
grid = Grid(init_opts = opts.InitOpts(theme=ThemeType.WONDERLAND,width='1200px',height='600px'))
grid.add(bar, opts.GridOpts(pos_left="5%", pos_right="20%"), is_control_axis_index=True)
grid.render_notebook()

### 代理维度分析回款发货

In [56]:
df_customer_returned_ship_day = df_returned_ship_03.groupby(['name_Alliance','AllianceName'])[['returned_money','ship_money']].sum().sort_values('returned_money',ascending = False)
df_customer_returned_ship_day.reset_index(inplace = True)
df_customer_returned_ship_day

,name_Alliance,AllianceName,returned_money,ship_money
0,毛文-黑马,黑马,152.0,153.5642
1,高朵朵-聚微,聚微,94.0,11.2066
2,刘双华-黑马,黑马,93.0,8.4280
3,郭孟洁-明星,明星,88.8,12.1208
4,王婷婷-风之彩,风之彩,88.0,32.9332
...,...,...,...,...
2915,宋彩娟-挑战者,挑战者,-4.0,0.0000
2916,张看看-满天星,满天星,-4.0,0.0000
2917,孙丽娜-满天星,满天星,-4.0,0.0000
2918,张丹-黑马,黑马,-5.0,7.6016


In [57]:
# df_customer_returned_ship_day.loc[df_customer_returned_ship_day['name_Alliance'].str.contains('郑国贤')]

#### 3月代理回款金额和均差额

In [58]:
df = diff(df_customer_returned_ship_day)[0:15]
df

,name_Alliance,AllianceName,returned_money,ship_money,diff_returned,diff_ship
0,毛文-黑马,黑马,152.0000,153.5642,147.814175,150.05571
1,高朵朵-聚微,聚微,94.0000,11.2066,89.814175,7.69811
2,刘双华-黑马,黑马,93.0000,8.4280,88.814175,4.91951
3,郭孟洁-明星,明星,88.8000,12.1208,84.614175,8.61231
4,王婷婷-风之彩,风之彩,88.0000,32.9332,83.814175,29.42471
5,刘亚歌-明星,明星,87.0000,22.9258,82.814175,19.41731
6,余宏敏-BOSS,BOSS,84.0000,29.6931,79.814175,26.18461
7,董全红-明星,明星,82.0110,23.3412,77.825175,19.83271
8,张雪萍-明星,明星,74.5376,18.1953,70.351775,14.68681
9,李燕-挑战者,挑战者,59.0000,58.7070,54.814175,55.19851


In [59]:
bar = (
    Bar()
    .add_xaxis(df['name_Alliance'].tolist())
    .extend_axis(yaxis=opts.AxisOpts(
        type_="value", position="right",axislabel_opts=opts.LabelOpts(font_size = 14,formatter="{value}"),
        axisline_opts = opts.AxisLineOpts(is_show = False)))
    .add_yaxis("回款较均值差额", round(df['diff_returned'],0).tolist(),yaxis_index=0)
    .add_yaxis("发货较均值差额", round(df['diff_ship'],0).tolist(),yaxis_index=0)
    .set_global_opts(
        title_opts=opts.TitleOpts(title="3月代理回款发货"),
        yaxis_opts = opts.AxisOpts(is_show = True,axislabel_opts=opts.LabelOpts(font_size = 14),axisline_opts = opts.AxisLineOpts(is_show = False)),
        xaxis_opts = opts.AxisOpts(axislabel_opts=opts.LabelOpts(font_size = 12,rotate = -45))
    )
    .set_series_opts(label_opts=opts.LabelOpts(font_size = 14,is_show = False))
                    )
line = (
    Line()
    .add_xaxis(df['name_Alliance'].tolist())
    .add_yaxis("回款金额",round(df['returned_money'],0).tolist() ,yaxis_index=1,z=10, markline_opts=opts.MarkLineOpts(data=[
                opts.MarkLineItem(type_="max", name="最大值"),
                opts.MarkLineItem(type_="average", name="平均值")
            ]))
    .add_yaxis("发货金额",round(df['ship_money'],0).tolist() ,yaxis_index=1,z=10, markpoint_opts=opts.MarkPointOpts(data=[opts.MarkPointItem(type_="max")]))
    .set_global_opts(
            title_opts =opts.TitleOpts(title = ''),
    )
    .set_series_opts(
            label_opts=opts.LabelOpts(font_size = 14,formatter= JsCode("function (params) {return params.value[1]+'%'}"),is_show= False)
    )        
)
bar.overlap(line)
grid = Grid(init_opts = opts.InitOpts(theme=ThemeType.WONDERLAND,width='1200px',height='600px'))
grid.add(bar, opts.GridOpts(pos_left="5%", pos_right="20%"), is_control_axis_index=True)
grid.render_notebook()

In [60]:
def func(s):
    if s >= 32:
        return '[32,+∞]'
    elif s>= 24:
        return '[24,32)'
    elif s>= 16:
        return '[16,24)'
    elif s>= 8:
        return '[8,16)' 
    elif s>= 4.25:
        return '[4.25,8)' 
    elif s> 0:
        return '(0,4.25)'
    else:
        return '其他' 

In [61]:
df_customer_returned_ship_day['return_interval'] = df_customer_returned_ship_day['returned_money'].apply(func)
df_customer_returned_ship_day

,name_Alliance,AllianceName,returned_money,ship_money,diff_returned,diff_ship,return_interval
0,毛文-黑马,黑马,152.0,153.5642,147.814175,150.05571,"[32,+∞]"
1,高朵朵-聚微,聚微,94.0,11.2066,89.814175,7.69811,"[32,+∞]"
2,刘双华-黑马,黑马,93.0,8.4280,88.814175,4.91951,"[32,+∞]"
3,郭孟洁-明星,明星,88.8,12.1208,84.614175,8.61231,"[32,+∞]"
4,王婷婷-风之彩,风之彩,88.0,32.9332,83.814175,29.42471,"[32,+∞]"
...,...,...,...,...,...,...,...
2915,宋彩娟-挑战者,挑战者,-4.0,0.0000,-8.185825,-3.50849,其他
2916,张看看-满天星,满天星,-4.0,0.0000,-8.185825,-3.50849,其他
2917,孙丽娜-满天星,满天星,-4.0,0.0000,-8.185825,-3.50849,其他
2918,张丹-黑马,黑马,-5.0,7.6016,-9.185825,4.09311,其他


In [62]:
df = df_customer_returned_ship_day.groupby(['AllianceName','return_interval'],as_index=False)['returned_money'].count()
df

,AllianceName,return_interval,returned_money
0,BOSS,"(0,4.25)",111
1,BOSS,"[16,24)",8
2,BOSS,"[32,+∞]",1
3,BOSS,"[4.25,8)",61
4,BOSS,"[8,16)",41
...,...,...,...
66,黑马,"[24,32)",2
67,黑马,"[32,+∞]",2
68,黑马,"[4.25,8)",54
69,黑马,"[8,16)",59


In [63]:
list_drop = df[df['return_interval'] == '其他'].index.tolist() + df[df['AllianceName'] == '其他'].index.tolist()

In [64]:
df.drop(list_drop,axis=0,inplace=True)


In [65]:
df.set_index('AllianceName',inplace=True)

In [66]:
df['rat_interval'] = df['returned_money'] /df['returned_money'].sum(level = 'AllianceName')
df.head()

,return_interval,returned_money,rat_interval
AllianceName,,,
BOSS,"(0,4.25)",111,0.500000
BOSS,"[16,24)",8,0.036036
BOSS,"[32,+∞]",1,0.004505
BOSS,"[4.25,8)",61,0.274775
BOSS,"[8,16)",41,0.184685


In [67]:
Alliance_return_interval = df.pivot_table(values='rat_interval',index='AllianceName',columns= 'return_interval').fillna(0)
Alliance_return_interval.sort_values('(0,4.25)',inplace = True)

In [68]:
Alliance_return_interval = Alliance_return_interval[['(0,4.25)','[4.25,8)','[8,16)','[16,24)','[24,32)','[32,+∞]']]
Alliance_return_interval

return_interval,"(0,4.25)","[4.25,8)","[8,16)","[16,24)","[24,32)","[32,+∞]"
AllianceName,,,,,,
风之彩,0.306667,0.240000,0.293333,0.093333,0.026667,0.040000
北极星,0.357143,0.285714,0.321429,0.035714,0.000000,0.000000
黑马,0.406250,0.241071,0.263393,0.071429,0.008929,0.008929
挑战者,0.432258,0.251613,0.270968,0.035484,0.006452,0.003226
聚微,0.476000,0.248000,0.252000,0.020000,0.000000,0.004000
BOSS,0.500000,0.274775,0.184685,0.036036,0.000000,0.004505
卓彩,0.503030,0.206061,0.242424,0.048485,0.000000,0.000000
女王,0.528662,0.222930,0.191083,0.044586,0.012739,0.000000
太阳,0.534247,0.260274,0.205479,0.000000,0.000000,0.000000


In [69]:
bar = (
    Bar(init_opts = opts.InitOpts(theme=ThemeType.WESTEROS,width='1000px',height='450px'))
    .add_xaxis(Alliance_return_interval.index.tolist())
    .extend_axis(yaxis=opts.AxisOpts(
        type_="value", position="right",axislabel_opts=opts.LabelOpts(font_size = 14,formatter="{value} %"),
        axisline_opts = opts.AxisLineOpts(is_show = False)))
    .add_yaxis("(0,4.25)", [round(n*100,2) for n in Alliance_return_interval.iloc[:,0].tolist()],yaxis_index=0,stack="stack1")
    .add_yaxis("[4.25,8)", [round(n*100,2) for n in Alliance_return_interval.iloc[:,1].tolist()],yaxis_index=0,stack="stack1")
    .add_yaxis("[8,16)", [round(n*100,2) for n in Alliance_return_interval.iloc[:,2].tolist()],yaxis_index=0,stack="stack1")
    .add_yaxis("[16,24)", [round(n*100,2) for n in Alliance_return_interval.iloc[:,3].tolist()],yaxis_index=0,stack="stack1")
    .add_yaxis("[24,32)", [round(n*100,2) for n in Alliance_return_interval.iloc[:,4].tolist()],yaxis_index=0,stack="stack1")
    .add_yaxis("[32,+∞]", [round(n*100,2) for n in Alliance_return_interval.iloc[:,5].tolist()],yaxis_index=0,stack="stack1")
    .reversal_axis()
    .set_global_opts(
        title_opts=opts.TitleOpts(title='联盟回款区间'),
        yaxis_opts = opts.AxisOpts(min_= 0,axislabel_opts=opts.LabelOpts(font_size = 14),axisline_opts = opts.AxisLineOpts(is_show = False)),
        xaxis_opts = opts.AxisOpts(axisline_opts = opts.AxisLineOpts(is_show = False),axislabel_opts=opts.LabelOpts(font_size = 12,rotate = 0,is_show= False)
    ))
    .set_series_opts(label_opts=opts.LabelOpts(font_size = 14,is_show=False))
                    )
bar.render_notebook()

### 联盟人均回款

In [70]:
df_AllianceName_returned = df_customer_returned_ship_day.groupby('AllianceName')['returned_money'].mean()
df_AllianceName_returned.sort_values(inplace =True,ascending = False)
df_AllianceName_returned.drop('其他',inplace=True)
df_AllianceName_returned

AllianceName
风之彩     6.704005
挑战者     5.318392
黑马      5.248468
北极星     4.328625
聚微      4.315237
女王      4.162112
卓彩      4.078461
BOSS    3.823824
明星      3.715775
太阳      2.848057
王者      2.576159
满天星     2.035385
Name: returned_money, dtype: float64

In [71]:
bar = (
    Bar(init_opts = opts.InitOpts(theme=ThemeType.WESTEROS,width='800px',height='450px'))
    .add_xaxis(df_AllianceName_returned.index.tolist())
    .add_yaxis("人均回款", round(df_AllianceName_returned,2).tolist())
    .set_global_opts(title_opts=opts.TitleOpts(title="3月联盟人均回款", subtitle=""),yaxis_opts = opts.AxisOpts(is_show= False))
    .set_series_opts(label_opts=opts.LabelOpts(font_size = 14),
                     markline_opts=opts.MarkLineOpts(data=[
                        opts.MarkLineItem(type_="average", name="平均值")],linestyle_opts  = opts.LineStyleOpts(color = 'green'))
    )
)
bar.render_notebook()

#### 客户回款区间

In [72]:
df_return_interval = df_customer_returned_ship_day.return_interval.value_counts()
df_return_interval.tolist()

[1069, 757, 518, 478, 77, 13, 8]

In [73]:
df_return_interval.index.tolist()

['(0,4.25)', '其他', '[4.25,8)', '[8,16)', '[16,24)', '[32,+∞]', '[24,32)']

In [74]:
Faker.values()

[74, 137, 84, 22, 78, 54, 106]

In [75]:
c = (
    Funnel(init_opts=opts.InitOpts(theme = ThemeType.WONDERLAND))
    .add("回款区间", [list(z) for z in zip(df_return_interval.index.tolist(), df_return_interval.tolist())],
        tooltip_opts=opts.TooltipOpts(trigger="item", formatter="{a} <br/>{b} : {c}"),
        label_opts=opts.LabelOpts(is_show=True, position="out",formatter = "{b}:{d}%"),
        itemstyle_opts=opts.ItemStyleOpts(border_color="#fff", border_width=1)
        
        )
    .set_global_opts(title_opts=opts.TitleOpts(title="3月回款区间"))
)
c.render_notebook()

### 官方维度当月回款发货

In [76]:
df_TopUserName_returned_ship_day = df_returned_ship_03.groupby('TopUserName')[['returned_money','ship_money']].sum()
df_TopUserName_returned_ship_day.reset_index(inplace = True)
df = diff(df_TopUserName_returned_ship_day)
df

,TopUserName,returned_money,ship_money,diff_returned,diff_ship
0,严小雅,131.5540,132.3668,1.872466,24.386789
1,付文姬,15.2600,25.1973,-114.421534,-82.782711
2,任娟娟,74.9335,61.6365,-54.748034,-46.343511
3,余丽,179.5067,146.9167,49.825166,38.936689
4,余宏敏,165.0495,103.6639,35.367966,-4.316111
...,...,...,...,...,...
89,颜叻,265.6706,205.6738,135.989066,97.693789
90,高倩倩,87.2198,73.8604,-42.461734,-34.119611
91,高朵朵,167.4986,56.4271,37.817066,-51.552911
92,魏娜,46.6100,42.3382,-83.071534,-65.641811


# 发货产品信息

In [77]:
sql_goods = f"""
"""

In [78]:
df_goods_info_03 = pd.read_sql(sql_goods,conn_bi)
df_goods_info_03.head()

ResourceClosedError: This result object does not return rows. It has been closed automatically.

In [165]:
df_spu_gy = df_goods_info_03.groupby('spu',as_index=False)[['money','totalCount']].sum()
df_spu_gy.head()

,spu,money,totalCount
0,BB霜,60.5492,10006
1,PS霜,49.2440,9758
2,七天面膜,3.6668,412
3,健康水,195.2319,20127
4,养护牙膏,12.0890,15861


In [166]:
df_spu_gy.sort_values(by='money',inplace=True,ascending=False)

In [167]:
df_spu_gy['sum_m'] = df_spu_gy.money.sum()
df_spu_gy['sum_c'] = df_spu_gy.totalCount.sum()

In [168]:
df_spu_gy['cunsum_m'] = df_spu_gy.money.cumsum()
df_spu_gy['cunsum_c'] = df_spu_gy.totalCount.cumsum()

In [169]:
df_spu_gy['rat_m'] = df_spu_gy['cunsum_m'] / df_spu_gy['sum_m']
df_spu_gy['rat_c'] = df_spu_gy['cunsum_c'] / df_spu_gy['sum_c']

In [375]:
df_spu_gy.reset_index(drop=True,inplace=True)
df_spu_gy.head()

,spu,money,totalCount,sum_m,sum_c,cunsum_m,cunsum_c,rat_m,rat_c
0,春水套盒,2966.5416,224743,9657.6809,1073841,2966.5416,224743,0.307169,0.209289
1,秋水套盒,1674.1560,126830,9657.6809,1073841,4640.6976,351573,0.480519,0.327398
2,高机能精华霜,868.7616,69504,9657.6809,1073841,5509.4592,421077,0.570474,0.392122
3,流光雪妍精萃水,538.3428,66857,9657.6809,1073841,6047.8020,487934,0.626217,0.454382
4,美白套盒,457.6176,34668,9657.6809,1073841,6505.4196,522602,0.673601,0.486666


In [194]:
bar = (
    Bar()
    .add_xaxis(df_spu_gy['spu'].tolist())
    .extend_axis(yaxis=opts.AxisOpts(
        max_=100,type_="value", position="right",axislabel_opts=opts.LabelOpts(font_size = 14,formatter="{value}%"),
        axisline_opts = opts.AxisLineOpts(is_show = False)))
    .add_yaxis("发货额", round(df_spu_gy['money'],0).tolist(),yaxis_index=0)
    .set_global_opts(
        title_opts=opts.TitleOpts(title="3月产品发货"),
        yaxis_opts = opts.AxisOpts(is_show = True,axislabel_opts=opts.LabelOpts(font_size = 14),axisline_opts = opts.AxisLineOpts(is_show = False)),
        xaxis_opts = opts.AxisOpts(axislabel_opts=opts.LabelOpts(font_size = 14,rotate = -90))
    )
    .set_series_opts(label_opts=opts.LabelOpts(font_size = 14,is_show = False))
                    )
line = (
    Line()
    .add_xaxis(df_spu_gy['spu'].tolist())
    .add_yaxis("发货额占比",[round(n*100,2) for n in df_spu_gy['rat_m'].tolist()],yaxis_index=1,z=10)
    .add_yaxis("发货数占比",[round(n*100,2) for n in df_spu_gy['rat_c'].tolist()],yaxis_index=1,z=10)
    .set_global_opts(
            title_opts =opts.TitleOpts(title = ''),
    )
    .set_series_opts(
            label_opts=opts.LabelOpts(font_size = 14,formatter= JsCode("function (params) {return params.value[1]+'%'}"),is_show= False)
    )        
)
overlap_1  = bar.overlap(line)
grid = (Grid(init_opts = opts.InitOpts(theme=ThemeType.WONDERLAND,width='1200px',height='400px'))
        .add(overlap_1, opts.GridOpts(pos_left="5%", pos_right="20%"), is_control_axis_index=True))
grid.render_notebook()

# 